Benötigte Module importieren und Datei laden. Die ersten Zeilen werden ausgegeben.

In [ ]:
import tensorflow as tf
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler

path = "../Data/adult-2.csv"
data = pd.read_csv(path, delimiter=';')
print(data.head())
print("Empty columns: ", data.columns[data.isnull().any()])

In [ ]:
# Ausgabe der Korrelationen
correlations = data[data.columns].corr(numeric_only=True)
#print(correlations)
print('All correlations')
print('-' * 30)
correlations_abs_sum = correlations[correlations.columns].abs().sum()
print(correlations_abs_sum)
print('Weakest correlations')
print('-' * 30)
print(correlations_abs_sum.nsmallest(5))

Daten vorbereiten.

In [ ]:
# Lösche drei Spalten mit kleinen Korrelationswerten
data.drop(['fnlwgt','capital.loss','capital.gain', 'race', 'sex', 'native.country'], axis = 1, inplace=True)

# income soll vorhergesagt werden
col = data['income']
col = pd.get_dummies(col, dtype=float)
data = data.drop(['income'], axis = 1)

# konvertiere diese Saplten in numerische Werte 0...n
conv_num = ['workclass', 'education', 'marital.status', 'occuaption','relationship', 'hours.per.week']
data[conv_num] = data[conv_num].astype('category')
data[conv_num] = data[conv_num].apply(lambda x: x.cat.codes)

# führe OHE für diese Daten durch
#conv_ohe = ['race', 'sex']
#data = pd.get_dummies(data, columns = conv_ohe, dtype=float)

#print(data.shape)
#print(data.shape[1])

# Erzeuge Objekte
s_scaler = StandardScaler()
m_scaler = MinMaxScaler()

# Spalten für StandardScaler
cols_to_s_scale = ['age', 'workclass', 'education', 'education.num', 'marital.status', 'occuaption', 'relationship']
data[cols_to_s_scale] = s_scaler.fit_transform(data[cols_to_s_scale])

# Spalten für MinMaxScaler
cols_to_m_scale = ['hours.per.week']
data[cols_to_m_scale] = m_scaler.fit_transform(data[cols_to_m_scale])

KNN aufbauen

In [ ]:
# Aus den zwei Tabellen vier Tabellen erzeugen
train_data, test_data, train_col, test_col = train_test_split(data,col, test_size=0.2, random_state=42)

# Aufbau KNN
model = tf.keras.Sequential([
    tf.keras.layers.Dense(128, activation=tf.nn.relu, input_dim=data.shape[1]),
    tf.keras.layers.Dense(256, activation=tf.nn.relu),
    tf.keras.layers.Dense(64, activation=tf.nn.relu),
    tf.keras.layers.Dense(2, activation=tf.nn.softmax)
])

# Konfiguration des Lernprozesses
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

Trainieren

In [ ]:
# 10 Durchläufe
model.fit(train_data, train_col, epochs=10)

Testen

In [ ]:
test_loss, test_acc = model.evaluate(test_data, test_col)
print('Test accuracy:', test_acc)